# Integrações com o CulinaryDB - Equipe GPALT (Analistas de Cardápios)

Os principais problemas com a base do CulinaryDB são que os espaços nos nomes das colunas apresentaram grandes dificuldades de integração. Mesmo tentando utilizar ``, o Jupyter não reconhecia esses dados. Assim, por simplificação, eles foram alterados.

### Receitas

Para o modelo proposto, o arquivo de receitas precisa ser ajustado. Isso pode ser feito em Python:

In [1]:
import csv

with open("../data/raw/culinarydb/01_Recipe_Details.csv", 'r') as original:
    with open("../data/interim/culinarydb/receitas.csv", 'w') as receitas:
        data = csv.reader(original)
        # Substituir o header
        next(data)
        print("IDReceita,Nome,BancoOriginal,Origem", file=receitas)
        for row in data:
            row[1] = row[1].replace("\"", "")
            print(f'"{row[0]}","{row[1]}","CulinaryDB","{row[3]}"', file=receitas)


### Ingredientes

Como o arquivo de associações montado considera os ingredientes originais com letras minúsculas, esse processamento será feito com os dados dos nomes dos ingredientes do CulinaryDB. Em relação aos próprios dados do CulinaryDB, o id dos ingredientes é necessário para as associações com as receitas e, então, deve ser mantido nessa etapa.

In [2]:
import csv

with open("../data/raw/culinarydb/02_Ingredients.csv", 'r') as original:
    with open("../data/interim/culinarydb/nomesingredients.csv", 'w') as ingredientes:
        data = csv.reader(original)
        # Substituir o header
        next(data)
        print("Nome,ID", file=ingredientes)
        for row in data:
            print(f'{row[0].lower()},{row[2]}', file=ingredientes)


### Ingredientes Compostos

Um problema com os ingredientes compostos originais é que o nome das colunas apresenta espaços. Esses espaços, para as consultas em SQL, atrapalham a importação dos dados. Removendo esses espaços (substituindo o header):

In [3]:
import pandas as pd

dados = pd.read_csv('../data/raw/culinarydb/03_Compound_Ingredients.csv')
colunas = ['Compound Ingredient Name','Compound Ingredient Synonyms','entity_id','Contituent Ingredients','Category']

with open("../data/interim/culinarydb/dadoscompostos.csv", 'w') as compostos:
    print("Compound_Ingredient_Name,Compound_Ingredient_Synonyms,entity_id,Contituent_Ingredients,Category", file=compostos)

    df = pd.DataFrame(dados, columns=colunas)
    df.to_csv(compostos, index=False, header=False)


Os ingredientes compostos da base são interessantes para receitas mais complexas. Segundo o modelo proposto, eles também devem estar na lista de ingredientes e também devem conter uma associação com os ingredientes simples que os formam. Como os ingredientes que formam os ingredientes compostos estão juntos em uma mesma linha, é possível separá-los:

In [4]:
import pandas as pd

dados = pd.read_csv('../data/raw/culinarydb/03_Compound_Ingredients.csv')

nomes = dados['Compound Ingredient Name'].values.tolist()
ingredientes = dados['Contituent Ingredients'].values.tolist()

with open("../data/interim/culinarydb/associacaocompostos.csv", 'w') as associacao:
    print("IngredienteComposto,IngredienteOriginal", file=associacao)

    # Para cada receita
    for i in range(len(nomes)):
        # Separar os nomes dos ingredientes
        lista = ingredientes[i].split(",")

        # Associações finais
        for k in lista:
            print(f'{nomes[i]},{k.strip()}', file=associacao)


### Ingredientes das Receitas

Em relação à associação de ingredientes e receitas, algumas alterações precisam ser feitas. Nos dados originais, algumas receitas têm quantidades associadas aos ingredientes que são úteis para as análises e, portanto, não podem ser descartados. O problema é que eles não são bem padronizados e estão juntos com o nome do ingrediente. Assim, o código Python a seguir foi elaborado para ajustar os dados aos modelos propostos, separando quantidade, unidade e fazendo cálculos de conversão de medidas:

In [5]:
import pandas as pd
from fractions import Fraction

dados = pd.read_csv('../data/raw/culinarydb/04_Recipe-Ingredients_Aliases.csv', error_bad_lines=False)

# Separando os dados em matrizes com os nomes dos ingredientes e quantidades, nome das receitas e ingrediente original da base
original = dados['Original Ingredient Name'].values.tolist()
rec_id = dados['Recipe ID'].values.tolist()
ingredientes_padrao = dados['Entity ID'].values.tolist()

# Matriz com os dados
ingre = []

# Dicionários com as palavras usadas com as unidades de medidas, e as outras com a conversão para unidades de medidas padrão
optional = ["to taste","own taste", "optional", "as needed", "garnish"]
unit_map = {"ounces": "ounce","ounce": "ounce","oz.": "ounce","oz": "ounce",
                "c.": "cup","c": "cup","C": "cup","cups": "cup","cup": "cup",
                "tbsp.": "tablespoon","tbsp": "tablespoon","tbs": "tablespoon","tablespoons": "tablespoon","tablespoon(s)": "tablespoon","tablespoon": "tablespoon",
                "teaspoons": "teaspoon","teaspoon": "teaspoon","tsp.": "teaspoon","tsp": "teaspoon","t.": "teaspoon",
                "lb.": "pond","lb": "pond","lbs": "pond","pound": "pond","pounds": "pond",
                "kg.": "kilogram","mg.": "miligram","grams": "gram","gram": "gram",
                "gm": "gram","g.": "gram","g": "gram",
                "qts": "quart","qts.": "quart","quarts": "quart","quart": "quart","qt.": "quart","qt": "quart",
                "gal.": "galon",
                "pints": "pint","pint": "pint","pt.": "pint",
                "liters": "liter","liter": "liter","l": "liter","ml": "mililiter",
                "inches": "inch","inch": "inch",
                "cm": "centimeter",
                "can": "can","cans": "can",
                "stick": "stick","sticks": "stick",
                "bottle": "bottle",
                "drops": "drop","drop": "drop",
               }
weight = {
        "gram": 0.001,
        "ounce" : 0.0283495,
        "pond" :  0.45359237,
        "kilogram" : 1,
    }
volumn = {
        "cup": 0.236588,
        "tablespoon" : 0.0147868,
        "teaspoon" : 0.00492892,
        "quart": 0.946353,
        "galon": 3.78541,
        "liters": 1,
        "mililiter": 0.001,
        "can": 0.3548824,
        "stick": 0.12,
        'pint': 0.473176,
    }

instructions= [", cut",", sliced",", drained", ', halved', ', for',', juiced',', dissolved', ', divided','- cut', ', cooked',
               '- rinsed',' for',', or',', any',', whipped','made from',', tied',', peeled',', seeded',', picked',', thawed',
               ' or','such as',', quartered',' plus',', lightly beaten',', woody',', each',', torn',', sifted']

way= [' chopped',' dried',' minced',' canned',' dry',' ground',' fresh',' finely', ' medium', ' raw','halves', 'skinless',
      'boneless','toasted','-inch','-thick','sliced','frozen','new','diced','sliced','Accompaniment: ','peeled','grated',
      'unpeeled','florets','small','at room temperature','to taste','room temperature','flat-leaf','uncooked','softened',
      'thinly','slivered','ripe','unbleached','firmly','melted','large']

# Limpa o texto excluindo os parenteses, retirando palavras sobre o modo de preapro e estado da comida, e
# por fim tranforma frações em números decimais
def limpa(texto):
    if '(' in texto and ')' in texto:
        st=texto.find('(')
        fm=texto.find(')')
        texto=texto.replace(str(texto[st-1:fm+1]),'')
    for a in range(len(instructions)):
        ind=texto.find(instructions[a])
        if ind!=-1:
            texto=texto[:ind]
    for a in range(len(way)):
        if way[a] in texto:
            texto= texto.replace(way[a],'')
    for val in texto.split():
        if '/' in val:
            ind=True
            for char in val:
                if not(char.isdigit() or char == '/'):
                    ind = False
            if ind==True:
                try:
                    novo=round(float(Fraction(val)),2)
                    texto=texto.replace(val,str(novo))
                except ValueError as e:
                    texto=texto.replace(val,'')
    return texto

# Converte as quantidades para valores padrão
def conversor(linha):
    mult=1
    # Quantidade é o item (exemplo, 1 ovo, 1 morango, ...)
    # ou não foi encontrada (linha[1] == 0)
    linha[2]='unidade'
    if linha[2] in weight:
        mult=weight[linha[2]]
        linha[2]='grama'
        linha[1]= round(1000 * mult * linha[1],3)
    elif linha[2] in volumn:
        mult=volumn[linha[2]]
        linha[2]='mililitro'
        linha[1]= round(1000 * mult * linha[1],3)
    return linha

# Passa pelas linhas fazendo a limpeza e separando os dados em quantidade, unidade de medida, comida e o id da receita
for a in range(len(original)):
    original[a]=limpa(original[a])
    num=0
    unit=''
    food=''
    original[a]=original[a].split()
    for b in range(len(original[a])):
        if any(char.isnumeric() for char in original[a][b]):
            try:
                num+=float(original[a][b])
            except ValueError as e:
                continue
        elif original[a][b] in unit_map:
            unit += " " + unit_map[original[a][b]]
        elif original[a][b] in optional:
            num=0
        else:
            food += ' ' + original[a][b]

    # Retira qualquer espaço ou vírgula sobrando
    unit=unit.strip().lstrip(',').rstrip(',')
    food=food.strip().lstrip(',').rstrip(',')
    # Id do ingrediente padrão reconhecido na base
    ingrediente_padrao = ingredientes_padrao[a]
    ingre.append(conversor([rec_id[a],num,unit,ingrediente_padrao]))


# Salva como banco de dados e faz o download como csv
columns = ['recipe_id', 'qtd', 'unit', 'ingredient']
df = pd.DataFrame(ingre, columns=columns)
df.insert(0, 'id_associacao', range(0, len(df)))
df.to_csv('../data/interim/culinarydb/ingredientesnasreceitas.csv', index=False)
